# retinal photography - start

In [ ]:
import pandas as pd
import pydicom


def get_retinal_photography_path(file):
    parts = file.split("/")

    # Find the index where 'retinal_photography' appears
    for i, part in enumerate(parts):
        if "retinal_photography" in part:
            new_path = "/".join(parts[i:])
            break

    return new_path


def get_dcm_files(folder_path):
    dcm_files = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            # exclude hidden/system files and ensure .dcm extension
            if not file.startswith(".") and file.lower().endswith(".dcm"):
                dcm_files.append(os.path.join(root, file))
    return dcm_files

# save metadata in json

In [ ]:
import json
import os


def save_retinal_photography_metadata_in_json(retinal_photography_folder):

    files = get_dcm_files(retinal_photography_folder)
    parts = retinal_photography_folder.rstrip("/").split("/")

    parts[-2] = parts[-2] + "_metadata"

    output_folder = "/".join(parts)

    for ir_file in files:

        dic = {
            "person_id": pydicom.dcmread(ir_file).PatientID,
            "manufacturer": "Heidelberg",
            "manufacturers_model_name": "Spectralis",
            "laterality": pydicom.dcmread(ir_file).ImageLaterality,
            "anatomic_region": "Macula, 20 x 20",
            "imaging": "Infrared Reflectance",
            "height": pydicom.dcmread(ir_file).Rows,
            "width": pydicom.dcmread(ir_file).Columns,
            "color_channel_dimension": "0",
            "sop_instance_uid": pydicom.dcmread(ir_file).SOPInstanceUID,
            "filepath": get_retinal_photography_path(ir_file),
        }

        filename = ir_file.split("/")[-1].replace(".", "_")

        json_data = {filename: dic}

        os.makedirs(f"{output_folder}", exist_ok=True)

        with open(f"{output_folder}/{filename}.json", "w") as json_file:
            json.dump(json_data, json_file)

    return output_folder

In [ ]:
retinal_photography_folder = "/Users/nayoonkim/Desktop/one_heidelberg/heidelberg_octa_imaging/retinal_photography"
metadata_folder = save_retinal_photography_metadata_in_json(retinal_photography_folder)

# make manifest

In [ ]:
import os
import json
import pandas as pd

folder_path = metadata_folder

rows = []

for fname in os.listdir(folder_path):
    if fname.startswith(".") or not fname.endswith(".json"):
        continue
    fpath = os.path.join(folder_path, fname)
    with open(fpath, "r") as f:
        data = json.load(f)
    for _, meta in data.items():
        rows.append(meta)

# Preserve JSON field order
df = pd.DataFrame(rows)

output_path = os.path.join(retinal_photography_folder, "manifest.tsv")
df.to_csv(output_path, sep="\t", index=False)

print(f"✅ TSV saved to: {output_path}")

# oct - start

In [ ]:
def get_retinal_oct_path(file):
    parts = file.split("/")

    # Find the index where 'retinal_photography' appears
    for i, part in enumerate(parts):
        if "retinal_oct" in part:
            new_path = "/".join(parts[i:])
            break

    return new_path


import pandas as pd


def find_filepath_by_uid(tsv_file, uid, uid_col="SOPInstanceUID", path_col="FilePath"):
    """
    Reads a TSV file and returns the 'FilePath' for matching 'SOPInstanceUID'.

    Parameters:
        tsv_file (str): path to the TSV file
        uid (str): UID to match
        uid_col (str): column name for UID
        path_col (str): column name for file path

    Returns:
        list[str]: list of matching file paths
    """
    df = pd.read_csv(tsv_file, sep="\t", dtype=str).fillna("")

    # Normalize column names to avoid case/underscore mismatches
    normalized = {c.lower().replace(" ", "").replace("_", ""): c for c in df.columns}
    n_uid = uid_col.lower().replace(" ", "").replace("_", "")
    n_fp = path_col.lower().replace(" ", "").replace("_", "")

    if n_uid not in normalized:
        raise KeyError(f"UID column '{uid_col}' not found. Columns: {list(df.columns)}")
    if n_fp not in normalized:
        raise KeyError(
            f"File path column '{path_col}' not found. Columns: {list(df.columns)}"
        )

    real_uid_col = normalized[n_uid]
    real_fp_col = normalized[n_fp]

    matches = df.loc[df[real_uid_col] == uid, real_fp_col].tolist()
    return matches[0]

# save retinal_oct metadata

In [ ]:
import json
import os


def save_retinal_oct_metadata_in_json(
    retinal_oct_folder, retinal_photography_manifest_tsv
):

    files = get_dcm_files(retinal_oct_folder)
    parts = retinal_oct_folder.rstrip("/").split("/")

    parts[-2] = parts[-2] + "_metadata"

    output_folder = "/".join(parts)

    for oct_file in files:

        dic = {
            "person_id": pydicom.dcmread(oct_file).PatientID,
            "manufacturer": "Heidelberg",
            "manufacturers_model_name": "Spectralis",
            "anatomic_region": "Macula, 20 x 20",
            "imaging": "OCT",
            "laterality": pydicom.dcmread(oct_file).ImageLaterality,
            "height": pydicom.dcmread(oct_file).Rows,
            "width": pydicom.dcmread(oct_file).Columns,
            "number_of_frames": pydicom.dcmread(oct_file).NumberOfFrames,
            "pixel_spacing": pydicom.dcmread(oct_file)[0x52009229][0][0x00289110][0][
                0x00280030
            ].value,
            "slice_thickness": pydicom.dcmread(oct_file)[0x52009229][0][0x00289110][0][
                0x00180050
            ].value,
            "sop_instance_uid": pydicom.dcmread(oct_file).SOPInstanceUID,
            "filepath": get_retinal_oct_path(oct_file),
            "reference_instance_uid": pydicom.dcmread(oct_file)[0x52009229][0][
                0x00081140
            ][0][0x00081155].value,
            "reference_filepath": find_filepath_by_uid(
                retinal_photography_manifest_tsv,
                pydicom.dcmread(oct_file)[0x52009229][0][0x00081140][0][
                    0x00081155
                ].value,
            ),
        }

        filename = oct_file.split("/")[-1].replace(".", "_")

        json_data = {filename: dic}

        os.makedirs(f"{output_folder}", exist_ok=True)

        with open(f"{output_folder}/{filename}.json", "w") as json_file:
            json.dump(json_data, json_file, default=str)

    return output_folder

In [ ]:
retinal_oct_files = (
    "/Users/nayoonkim/Desktop/one_heidelberg/heidelberg_octa_imaging/retinal_oct"
)
retinal_photography_manifest = "/Users/nayoonkim/Desktop/one_heidelberg/heidelberg_octa_imaging/retinal_photography/manifest.tsv"
metadata_folder = save_retinal_oct_metadata_in_json(
    retinal_oct_files, retinal_photography_manifest
)

# make manifest

In [ ]:
import os
import json
import pandas as pd

folder_path = metadata_folder

rows = []

for fname in os.listdir(folder_path):
    if fname.startswith(".") or not fname.endswith(".json"):
        continue
    fpath = os.path.join(folder_path, fname)
    with open(fpath, "r") as f:
        data = json.load(f)
    for _, meta in data.items():
        rows.append(meta)

# Preserve JSON field order
df = pd.DataFrame(rows)

output_path = os.path.join(retinal_oct_files, "manifest.tsv")
df.to_csv(output_path, sep="\t", index=False)

print(f"✅ TSV saved to: {output_path}")

# octa - start

In [15]:
import os


def get_dcm_files(folder_path):
    dcm_files = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            # exclude hidden/system files and ensure .dcm extension
            if not file.startswith(".") and file.lower().endswith(".dcm"):
                dcm_files.append(os.path.join(root, file))
    return dcm_files


def get_retinal_octa_path(file):
    parts = file.split("/")

    # Find the index where 'retinal_photography' appears
    for i, part in enumerate(parts):
        if "retinal_octa" in part:
            new_path = "/".join(parts[i:])
            break

    return new_path


import os


def get_json_flow_files(root_folder):
    """
    Walk through a folder and collect all JSON files that:
      - end with '.json'
      - do NOT start with '.'
      - include 'flow' in the filename
    """
    matches = []
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            if (
                file.endswith(".json")
                and not file.startswith(".")
                and "flow" in file.lower()
            ):
                matches.append(os.path.join(root, file))
    return matches


def get_json_enface_files(root_folder):
    """
    Walk through a folder and collect all JSON files that:
      - end with '.json'
      - do NOT start with '.'
      - include 'flow' in the filename
    """
    matches = []
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            if (
                file.endswith(".json")
                and not file.startswith(".")
                and "enface" in file.lower()
            ):
                matches.append(os.path.join(root, file))
    return matches


def get_json_segmentation_files(root_folder):
    """
    Walk through a folder and collect all JSON files that:
      - end with '.json'
      - do NOT start with '.'
      - include 'flow' in the filename
    """
    matches = []
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            if (
                file.endswith(".json")
                and not file.startswith(".")
                and "segmentation" in file.lower()
            ):
                matches.append(os.path.join(root, file))
    return matches


def find_filepath_by_uid(tsv_file, uid, uid_col="SOPInstanceUID", path_col="FilePath"):
    """
    Reads a TSV file and returns the 'FilePath' for matching 'SOPInstanceUID'.

    Parameters:
        tsv_file (str): path to the TSV file
        uid (str): UID to match
        uid_col (str): column name for UID
        path_col (str): column name for file path

    Returns:
        list[str]: list of matching file paths
    """
    df = pd.read_csv(tsv_file, sep="\t", dtype=str).fillna("")

    # Normalize column names to avoid case/underscore mismatches
    normalized = {c.lower().replace(" ", "").replace("_", ""): c for c in df.columns}
    n_uid = uid_col.lower().replace(" ", "").replace("_", "")
    n_fp = path_col.lower().replace(" ", "").replace("_", "")

    if n_uid not in normalized:
        raise KeyError(f"UID column '{uid_col}' not found. Columns: {list(df.columns)}")
    if n_fp not in normalized:
        raise KeyError(
            f"File path column '{path_col}' not found. Columns: {list(df.columns)}"
        )

    real_uid_col = normalized[n_uid]
    real_fp_col = normalized[n_fp]

    matches = df.loc[df[real_uid_col] == uid, real_fp_col].tolist()
    return matches[0]

In [16]:
import json
import pydicom


def save_octa_metadata_in_json(retinal_octa_folder):

    files = get_dcm_files(retinal_octa_folder)
    parts = retinal_octa_folder.rstrip("/").split("/")

    parts[-2] = parts[-2] + "_metadata"

    output_folder = "/".join(parts)

    for f in files:

        if pydicom.dcmread(f).SOPClassUID == "1.2.840.10008.5.1.4.1.1.77.1.5.8":
            # print("vol")

            vol = pydicom.dcmread(f)

            dic = {
                "participant_id": vol.PatientID,
                "filepath": get_retinal_octa_path(f),
                "modality": "octa",
                "submodality": "flow_cube",
                "manufacturer": "Heidelberg",
                "manufacturers_model_name": "Spectralis",
                "laterality": vol.Laterality,
                "protocol": vol.ProtocolName,
                "height": vol.Rows,
                "width": vol.Columns,
                "number_of_frames": vol.NumberOfFrames,
                "sop_instance_uid": vol.SOPInstanceUID,
                "reference_oct_uid": vol[0x00081115][1][0x0008114A][0][
                    0x00081155
                ].value,
                "reference_op_uid": vol[0x00081115][0][0x0008114A][0][0x00081155].value,
                "content_time": vol.ContentDate + vol.ContentTime,
                "sop_class_uid": vol.SOPClassUID,
            }
            # print(dic)

            filename = f.split("/")[-1].replace(".", "_")

            json_data = {filename: dic}

            os.makedirs(f"{output_folder}", exist_ok=True)

            with open(f"{output_folder}/{filename}.json", "w") as json_file:
                json.dump(json_data, json_file)

        elif pydicom.dcmread(f).SOPClassUID == "1.2.840.10008.5.1.4.1.1.77.1.5.7":
            # print("enface")

            enface = pydicom.dcmread(f)

            dic = {
                "participant_id": enface.PatientID,
                "filepath": get_retinal_octa_path(f),
                "modality": "octa",
                "submodality": "enface",
                "manufacturer": "Heidelberg",
                "manufacturers_model_name": "Spectralis",
                "laterality": enface.ImageLaterality,
                "protocol": enface.ProtocolName,
                "height": enface.Rows,
                "width": enface.Columns,
                "sop_instance_uid": enface.SOPInstanceUID,
                "ophthalmic_image_type": enface[0x00221615][0][0x00080104].value,
                "En Face Retinal Segmentation Surface 1": str(
                    enface["00221627"][0]["0008114C"][0]["0062000F"][0][
                        "00080104"
                    ].value
                ).lower(),
                "En Face Retinal Segmentation Surface 2": str(
                    enface["00221627"][1]["0008114C"][0]["0062000F"][0][
                        "00080104"
                    ].value
                ).lower(),
                "content_time": enface.ContentDate + enface.ContentTime,
                "sop_class_uid": enface.SOPClassUID,
                "oct_reference_instance_uid": enface[0x00081115][1][0x0008114A][0][
                    0x00081155
                ].value,
                "op_reference_instance_uid": enface[0x00081115][0][0x0008114A][0][
                    0x00081155
                ].value,
                "vol_reference_instance_uid": enface[0x00081115][2][0x0008114A][0][
                    0x00081155
                ].value,
                "seg_reference_instance_uid": enface[0x00081115][3][0x0008114A][0][
                    0x00081155
                ].value,
            }

            # print(dic)

            filename = f.split("/")[-1].replace(".", "_")

            json_data = {filename: dic}

            os.makedirs(f"{output_folder}", exist_ok=True)

            with open(f"{output_folder}/{filename}.json", "w") as json_file:
                json.dump(json_data, json_file)

        elif pydicom.dcmread(f).SOPClassUID == "1.2.840.10008.5.1.4.1.1.66.8":

            seg = pydicom.dcmread(f)
            raw_alg = seg[0x00620002][0][0x00620009].value if 0x00620002 in seg else ""
            alg_name = (str(raw_alg) or "").lower()

            segmentation_algorithm_type = (
                "smoothed" if "smoothed" in alg_name else "not smoothed"
            )

            dic = {
                "participant_id": seg.PatientID,
                "filepath": get_retinal_octa_path(f),
                "modality": "octa",
                "submodality": "segmentation",
                "manufacturer": "Heidelberg",
                "manufacturers_model_name": "Spectralis",
                "laterality": seg.Laterality,
                "protocol": seg.ProtocolName,
                "height": seg.Rows,
                "width": seg.Columns,
                "number_of_frames": seg.NumberOfFrames,
                "sop_instance_uid": seg.SOPInstanceUID,
                "oct_reference_instance_uid": seg[0x00081115][1][0x0008114A][0][
                    0x00081155
                ].value,
                "segmentation_type": "Heightmap",
                "segmentation_algorithm_type": segmentation_algorithm_type,
                "content_time": seg.ContentDate + seg.ContentTime,
                "sop_class_uid": seg.SOPClassUID,
            }

            # print("seg")
            # print(dic)

            filename = f.split("/")[-1].replace(".", "_")

            json_data = {filename: dic}

            os.makedirs(f"{output_folder}", exist_ok=True)

            with open(f"{output_folder}/{filename}.json", "w") as json_file:
                json.dump(json_data, json_file)

    return output_folder

In [ ]:
metadata_folder = save_octa_metadata_in_json(
    "/Users/nayoonkim/Desktop/one_heidelberg/heidelberg_octa_imaging/retinal_octa"
)
metadata_folder

# make manifest

In [18]:
def make_octa_manifest(
    folder, retinal_photography_manifest_tsv, retinal_oct_manifest_tsv
):

    flow_json_files = get_json_flow_files(folder)

    for flow in flow_json_files:

        with open(flow, "r") as f:

            data = json.load(f)
            person_id = next(iter(data.values()))["participant_id"]
            manufacturer = next(iter(data.values()))["manufacturer"]

            manufacturers_model_name = next(iter(data.values()))[
                "manufacturers_model_name"
            ]
            anatomic_region = "Macula, 20 x 20"
            imaging = "OCTA"
            laterality = next(iter(data.values()))["laterality"]

            flow_cube_sop_instance_uid = next(iter(data.values()))["sop_instance_uid"]
            flow_cube_height = next(iter(data.values()))["height"]
            flow_cube_width = next(iter(data.values()))["width"]
            flow_cube_number_of_frames = next(iter(data.values()))["number_of_frames"]
            flow_cube_file_path = next(iter(data.values()))["filepath"]

            associated_retinal_photography_sop_instance_uid = next(iter(data.values()))[
                "reference_op_uid"
            ]
            associated_structural_oct_sop_instance_uid = next(iter(data.values()))[
                "reference_oct_uid"
            ]

        enface_json_files = get_json_enface_files(folder)

        matching_enface_files = []
        for enface_file in enface_json_files:
            with open(enface_file, "r") as f:
                data = json.load(f)
                inner = next(iter(data.values()), {})
                if (
                    inner.get("vol_reference_instance_uid")
                    == flow_cube_sop_instance_uid
                ):
                    matching_enface_files.append(enface_file)

        for matching_enface in matching_enface_files:

            with open(matching_enface, "r") as enface:
                data = json.load(enface)
                if (
                    next(iter(data.values()))["ophthalmic_image_type"]
                ) == "Superficial retina vasculature flow":
                    associated_enface_1_ophthalmic_image_type = next(
                        iter(data.values())
                    )["ophthalmic_image_type"]
                    associated_enface_1_segmentation_surface_1 = next(
                        iter(data.values())
                    )["En Face Retinal Segmentation Surface 1"]
                    associated_enface_1_segmentation_surface_2 = next(
                        iter(data.values())
                    )["En Face Retinal Segmentation Surface 2"]
                    associated_enface_1_sop_instance_uid = next(iter(data.values()))[
                        "sop_instance_uid"
                    ]
                    associated_enface_1_file_path = next(iter(data.values()))[
                        "filepath"
                    ]

                    print(associated_enface_1_ophthalmic_image_type)
                    print(associated_enface_1_segmentation_surface_1)
                    print(associated_enface_1_segmentation_surface_2)
                    print(associated_enface_1_sop_instance_uid)
                    print(associated_enface_1_file_path)

                elif (
                    next(iter(data.values()))["ophthalmic_image_type"]
                ) == "Deep retina vasculature flow":
                    associated_enface_2_ophthalmic_image_type = next(
                        iter(data.values())
                    )["ophthalmic_image_type"]
                    associated_enface_2_segmentation_surface_1 = next(
                        iter(data.values())
                    )["En Face Retinal Segmentation Surface 1"]
                    associated_enface_2_segmentation_surface_2 = next(
                        iter(data.values())
                    )["En Face Retinal Segmentation Surface 2"]
                    associated_enface_2_sop_instance_uid = next(iter(data.values()))[
                        "sop_instance_uid"
                    ]
                    associated_enface_2_file_path = next(iter(data.values()))[
                        "filepath"
                    ]
                    associated_enface_2_projection_removed_sop_instance_uid = (
                        "Not Reported"
                    )
                    associated_enface_2_projection_removed_filepath = "Not Reported"

                    print(associated_enface_2_ophthalmic_image_type)
                    print(associated_enface_2_segmentation_surface_1)
                    print(associated_enface_2_segmentation_surface_2)
                    print(associated_enface_2_sop_instance_uid)
                    print(associated_enface_2_file_path)
                    print(associated_enface_2_projection_removed_sop_instance_uid)
                    print(associated_enface_2_projection_removed_filepath)

                elif (
                    next(iter(data.values()))["ophthalmic_image_type"]
                ) == "Avascular complex flow":
                    associated_enface_4_ophthalmic_image_type = next(
                        iter(data.values())
                    )["ophthalmic_image_type"]
                    associated_enface_4_segmentation_surface_1 = next(
                        iter(data.values())
                    )["En Face Retinal Segmentation Surface 1"]
                    associated_enface_4_segmentation_surface_2 = next(
                        iter(data.values())
                    )["En Face Retinal Segmentation Surface 2"]
                    associated_enface_4_sop_instance_uid = next(iter(data.values()))[
                        "sop_instance_uid"
                    ]
                    associated_enface_4_file_path = next(iter(data.values()))[
                        "filepath"
                    ]
                    associated_enface_4_projection_removed_sop_instance_uid = (
                        "Not Reported"
                    )
                    associated_enface_4_projection_removed_filepath = "Not Reported"

                    print(associated_enface_4_ophthalmic_image_type)
                    print(associated_enface_4_segmentation_surface_1)
                    print(associated_enface_4_segmentation_surface_2)
                    print(associated_enface_4_sop_instance_uid)
                    print(associated_enface_4_file_path)
                    print(associated_enface_4_projection_removed_sop_instance_uid)
                    print(associated_enface_4_projection_removed_filepath)

        segmentation_json_files = get_json_segmentation_files(folder)

        matching_segmentation_files = []
        for segmentation_file in segmentation_json_files:
            with open(segmentation_file, "r") as f:
                data = json.load(f)
                inner = next(iter(data.values()), {})
                if (
                    inner.get("oct_reference_instance_uid")
                    == associated_structural_oct_sop_instance_uid
                ):
                    matching_segmentation_files.append(segmentation_file)

        for matching_segmentation in matching_segmentation_files:

            with open(matching_segmentation, "r") as segmentation:
                data = json.load(segmentation)

                if (
                    next(iter(data.values()))["segmentation_algorithm_type"]
                    == "smoothed"
                ):

                    associated_segmentation_type = "Heightmap"

                    associated_segmentation_number_of_frames = next(
                        iter(data.values())
                    )["number_of_frames"]
                    associated_segmentation_sop_instance_uid = next(
                        iter(data.values())
                    )["sop_instance_uid"]
                    associated_segmentation_file_path = next(iter(data.values()))[
                        "filepath"
                    ]

                    print(associated_segmentation_number_of_frames)
                    print(associated_segmentation_sop_instance_uid)
                    print(associated_segmentation_file_path)

                elif (
                    next(iter(data.values()))["segmentation_algorithm_type"]
                    == "not smoothed"
                ):
                    variant_segmentation_sop_instance_uid = next(iter(data.values()))[
                        "sop_instance_uid"
                    ]
                    variant_segmentation_file_path = next(iter(data.values()))[
                        "filepath"
                    ]

                    print(variant_segmentation_sop_instance_uid)
                    print(variant_segmentation_file_path)

        associated_enface_3_ophthalmic_image_type = "Not Reported"
        associated_enface_3_segmentation_surface_1 = "Not Reported"
        associated_enface_3_segmentation_surface_2 = "Not Reported"
        associated_enface_3_sop_instance_uid = "Not Reported"
        associated_enface_3_file_path = "Not Reported"
        associated_enface_3_projection_removed_sop_instance_uid = "Not Reported"
        associated_enface_3_projection_removed_filepath = "Not Reported"
        associated_retinal_photography_file_path = find_filepath_by_uid(
            retinal_photography_manifest_tsv,
            associated_retinal_photography_sop_instance_uid,
        )
        associated_structural_oct_file_path = find_filepath_by_uid(
            retinal_oct_manifest_tsv, associated_structural_oct_sop_instance_uid
        )

        one_row = [
            person_id,
            manufacturer,
            manufacturers_model_name,
            anatomic_region,
            imaging,
            laterality,
            flow_cube_height,
            flow_cube_width,
            flow_cube_number_of_frames,
            associated_segmentation_type,
            associated_segmentation_number_of_frames,
            associated_enface_1_ophthalmic_image_type,
            associated_enface_1_segmentation_surface_1,
            associated_enface_1_segmentation_surface_2,
            associated_enface_2_ophthalmic_image_type,
            associated_enface_2_segmentation_surface_1,
            associated_enface_2_segmentation_surface_2,
            associated_enface_3_ophthalmic_image_type,
            associated_enface_3_segmentation_surface_1,
            associated_enface_3_segmentation_surface_2,
            associated_enface_4_ophthalmic_image_type,
            associated_enface_4_segmentation_surface_1,
            associated_enface_4_segmentation_surface_2,
            flow_cube_sop_instance_uid,
            flow_cube_file_path,
            associated_retinal_photography_sop_instance_uid,
            associated_retinal_photography_file_path,
            associated_structural_oct_sop_instance_uid,
            associated_structural_oct_file_path,
            associated_segmentation_sop_instance_uid,
            associated_segmentation_file_path,
            variant_segmentation_sop_instance_uid,
            variant_segmentation_file_path,
            associated_enface_1_sop_instance_uid,
            associated_enface_1_file_path,
            associated_enface_2_sop_instance_uid,
            associated_enface_2_file_path,
            associated_enface_2_projection_removed_sop_instance_uid,
            associated_enface_2_projection_removed_filepath,
            associated_enface_3_sop_instance_uid,
            associated_enface_3_file_path,
            associated_enface_3_projection_removed_sop_instance_uid,
            associated_enface_3_projection_removed_filepath,
            associated_enface_4_sop_instance_uid,
            associated_enface_4_file_path,
            associated_enface_4_projection_removed_sop_instance_uid,
            associated_enface_4_projection_removed_filepath,
        ]

        print(one_row)
        print(len(one_row))

In [ ]:
import manifest_heidelberg_octa_function_test


folder = "/Users/nayoonkim/Desktop/one_heidelberg/heidelberg_octa_imaging_metadata/retinal_octa"
op = "/Users/nayoonkim/Desktop/one_heidelberg/heidelberg_octa_imaging/retinal_photography/manifest.tsv"
opt = "/Users/nayoonkim/Desktop/one_heidelberg/heidelberg_octa_imaging/retinal_oct/manifest.tsv"
make_octa_manifest(folder, op, opt)